In [1]:
# A CITY REGION IN MAKING - DEMO VERSION
# AUTHOR: DR LI WAN | UNIVERSITY OF CAMBRIDGE
# ACKNOWLEDGEMENTS: MS TIANYUAN WANG FOR CODING ASSISTANCE

import os
import scipy.io as sio
import pandas as pd
import numpy as np
import time

## DATA INPUT

In [2]:
## Model Parameters
MaxITN = 5000         # max iteration times
Tol = 1e-6            # tolerance for model convergence
Status_Mode = 0       # 1: Calibdation mode; 0: Forecast mode
Status_EmpPred = 1    # 1: Predict emp-residential location pair; 0: Predict residential location only 
Status_HrentPred = 1  # 1: Endogenous house rents; 0: Exogenous house rents
LLCoefIJ = np.array([[0.0,0.0]]) # log-linear transformation coef
D = 250               # number of working days
Lambda = np.array([[1.0,1.0]])  # dispersion parameter for location choice? (can't see those words after location)
LT = len(Lambda[0])   # number of labour type

In [3]:
## Model Vairables

# total employment by socio-economic classification; input if Status_EmpPred == 1
# left-hand side is high-income group; right-hand side is low-income group
EmpSeCTot = np.array([[300,1]]) 

# employment by socio-economic classification; input if Status_EmpPred == 0
# left-hand side is high-income group; right-hand side is low-income group
EmpSeC = np.array([[100,1],   
                   [100,1],
                   [100,1]])

# define employment input
if Status_EmpPred == 1:
    EmpInput = EmpSeCTot
else:
    EmpInput = EmpSeC

# travel time from residence place (row) to work place (column)(Unit: minute)
# For example: 5 minutes travelling from zone 1 as residence to zone 1 to work
Time1 = np.array([[5,15,30],    
                  [15,5,15],
                  [30,15,5]]) 

# assuming same travel time for all SeCs
Time = np.repeat(Time1[None,...],LT,axis=0)

Dist = Time # travel distance matrix (Unit: km)

# housing floorspace (Unit: m2) - from top to bottom, zone 1, 2, 3 respectively  
HS = np.array([[1000],
               [1000],
               [1000]])

# business floorspace (Unit: m2)
BFS = np.array([[1000],
                [1000],
                [1000]])      # business floorspace

# house rent per m2
Hrent0 = np.array([[200],
                   [200],
                   [200]])    # unit house rent

Hrent = Hrent0

# annual labour wage at worlplace
# left-hand side is high-income group; right-hand side is low-income group
Wage = np.array([[10000,10000],   
                 [10000,10000],
                 [10000,10000]])

# Share of total income spent on housing
# left-hand side is high-income group; right-hand side is low-income group
HSExpShare = np.array([[0.2,0.2],   
                       [0.2,0.2],
                       [0.2,0.2]])

# number of zones - read from housing input
ZNum = len(HS)

In [4]:
# read zonal residual attractiveness term from file (saved on server)
name_ZAT = 'ZAT'
name_ZAttrI = 'ZAttrI'
name_ZAttrIJ = 'ZAttrIJ'

from functions import read_ZAT
ZAttrI, ZAttrIJ = read_ZAT(LT,ZNum,name_ZAT,name_ZAttrI,name_ZAttrIJ) 
# (cuz this function needs ZNum variable as well, I put it after generating ZNum variable? Otherwise cannot run actually..)

------------------- ZAT file exists - Load ZAT file -----------------


## MODEL ITERATIONS

In [5]:
from functions import ProbIJ_Mix, Update_Hrent, Calibrate_ZAttr

start_time = time.time()

if Status_HrentPred == 1:
    print('--------------------------- Iteration starts ------------------------')
    
    for k in list(range(1,MaxITN+1)):
        
        if k == MaxITN:
            print('-------------------------- MaxITN reached --------------------------')
            break
        
        Output = ProbIJ_Mix(Status_EmpPred,D,LLCoefIJ,Lambda,EmpInput,Time,Dist,HS,BFS,Hrent0,ZAttrIJ,ZAttrI, LT,ZNum) #add LT,ZNum
        Hrent, Error = Update_Hrent(Output, LT,ZNum,Wage,HSExpShare,Hrent0,HS)
        
        if Error < Tol:
            print('--------------------- Hrent Converged at ITN = {} ------------------'.format(k))
            break
        else:
            Hrent0 = 1.0*Hrent + 0.0*Hrent0
            continue
    
else:
    print('--------------- Calculate location choice probability ---------------')
    Output = ProbIJ_Mix(Status_EmpPred,D,LLCoefIJ,Lambda,EmpInput,Time,Dist,HS,BFS,Hrent0,ZAttrIJ,ZAttrI, LT,ZNum)
    

if Status_Mode == 1:
    print('---------------------- ZATTR Calibration start ----------------------')
    ZAttrIJ,ZAttrI = Calibrate_ZAttr(D,LLCoefIJ,Lambda,Time,HS,BFS,Hrent, LT,ZNum)
    sio.savemat('ZAT(Python).mat', {'ZAttrIJ':ZAttrIJ, 'ZAttrI':ZAttrI})

print("Elapsed time is: %.4f seconds" % (time.time() - start_time)) 

--------------------------- Iteration starts ------------------------
--------------------- Hrent Converged at ITN = 1832 ------------------
Elapsed time is: 0.8800 seconds


## # MODEL OUTPUT

In [6]:
from functions import print_outputs
Output_summary = print_outputs (Status_Mode,Status_EmpPred,Status_HrentPred,Output,Hrent,Tol) # run this function
Output_summary.keys()  # display all the table names in the 'output_summary' file

dict_keys(['Metadata', 'MetadataT', 'T_IJ', 'T_IJ_all', 'T_EREW', 'T_Hrents', 'T_JobOppLatCat', 'T_Tran'])

In [7]:
Output_summary['Metadata']  # there is a bit of date format issue - I will check it later. 

[['PROJECT NAME: ProbIJ_Model_Test'],
 ['DATE: ', Timestamp('2021-08-19 19:38:35.609619')],
 ['AUTHOR: LI WAN | UNIVERSITY OF CAMBRIDGE'],
 ['PRECISION: ', 1e-06],
 ['MODEL MODE: FORECAST'],
 ['EMPLOTMENT PREDICTION: ENABLED'],
 ['HOUSE RENTS PREDICTION: ENABLED']]

In [8]:
Output_summary['T_EREW'] # select one table to display. (just change this table name to what we're interested in)

ER                    EW          
     Column_A  Column_B    Column_A  Column_B
0  103.672466  0.345575   89.494273  0.298314
1   92.655067  0.308850  121.011454  0.403372
2  103.672466  0.345575   89.494273  0.298314

In [9]:
Output_summary['T_Hrents'] 

,Hrent
0,208.035876
1,185.927687
2,208.035876


In [10]:
Output_summary['T_IJ'] 

dim_column            0          1          2
dim3 dim_row                                 
0    0        59.241409  29.620705  14.810352
     1        15.442511  61.770045  15.442511
     2        14.810352  29.620705  59.241409
1    0         0.197471   0.098736   0.049368
     1         0.051475   0.205900   0.051475
     2         0.049368   0.098736   0.197471

In [11]:
Output_summary['T_IJ_all'] 

,0,1,2
0,59.438881,29.719440,14.859720
1,15.493986,61.975945,15.493986
2,14.859720,29.719440,59.438881


In [12]:
# # check carlibration 
# sio.loadmat('ZAT(Python).mat')['ZAttrI']

# # compare results generated from Matlab
# matZAT = sio.loadmat('Simplified_Matlab_Model_v3_Calibration/ZAT.mat')['ZAT']
# ZAT = matZAT[0,0]    # ZAT.dtype
# ZAttrI = np.moveaxis(ZAT['ZAttrI'], -1, 0)
# ZAttrIJ = np.moveaxis(ZAT['ZAttrIJ'], -1, 0)

# ZAttrI